# AI PROJECT_3 SARA TAVAKOLI 810196684

موضوع پروژه : پردازش متن و شبکه های بیزین
در این پروژه می خواهیم یک سیستم دسته بندی موضوع اخبار بسازیم و به کمک آن موضوع خبر هایی که به ما داده شده است را تعیین کنیم. برای این کار ابتدا دسته ای از داده هایی که به ما داده شده است (فایل دیتا) استفاده می کنیم تا به کمک آن ها اخبار های موجود در هر دسته را کاوش کنیم و احتمال هر کلمه در آن اخبار را به کمک شبکه بیزین به دست آوریم و سیستم خود را ترین کنیم. 
برای ارزیابی این سیستم از بخش دیگری از داده ها استفاده می کنیم تا میزان درستی حدس خود را بسنجیم. و در نهایت سیستم ساخته شده را روی داده های تست اعمال می کنیم و دسته ی آن ها را مشخص می کنیم.

In [ ]:
import pandas as pd
import nltk
import re
import math
import sys
import random

FEILD = 'short_description'
FILELOCATION = "Attachment/data.csv"
TESTFILELOCATION = "Attachment/test.csv"
CATEGORYCOL = 'category'
BUSINESSCATEGORY = 'BUSINESS'
STYLEANDBEAUTYCATEGORY = 'STYLE & BEAUTY'
TRAVELCATEGORY = 'TRAVEL'

تمیز کردن داده:
ابتدا باید متن مربوط به هر خبر را تمیز کنیم.
برای این کار مراحل زیر را انجام می دهیم:
nanابتدا سطر هایی که ستون مورد نظر آن ها 
باشد را حذف  میکنیم

سپس تمام حروف متن را به حروف کوچک تبدیل می کنیم. زیرا بزرگ بودن یک حرف در کلمه تباید باعث شود که آن را متمایز از حالتی بگیریم که یک یا چند حرف در آن کلمه بزرگ باشند

سپس کلماتی مانند ضمیر های اشاره و شمیر های شخص و ... را جذف می کنیم زیرا این کلمات در تشخیص نوع خبر به ما کمکی نمیکنند و تفاوتی ایجاد نمی شود.

سپس تمام علائم نگارشی و اعداد را نیز به دلیل مفید نبودن و بی تاثیر بودن در تشخیض حذف می کنیم

بعد از آن با استفاده از روش stemming
کلمات را با ریشه ه آن ها جایگزین می کنیم تا به ازای همه ی آن ها یک کلمه داشته باشیم. زیرا در واقع اگر هر حالتی از یک فعل در متن آمده باشد در واقع بیان گر ریشه همان فعل است و نیازس به داشتن تمام آن حالت ها در دیکشنری هر دسته خبر نداریم.





In [ ]:
def readData(fileLoc):
    return pd.read_csv(fileLoc)

def removeRowsWithNanDescription(data, textField):
    data = data[data['short_description'].notnull()]
    return data

def changeUpperCaseCharsWithLower(data, textField):
    data[textField] = data[textField].str.lower()
    return data

def removeSymbolsAnsNums(data, textField):
    data[textField] = data[textField].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    data[textField] = data[textField].apply(lambda elem: re.sub(r"\d+", "", elem))
    return data

def cleanText(data, textField):
    data = changeUpperCaseCharsWithLower(data, textField)
    data = removeSymbolsAnsNums(data, textField)
    return data

def removeStopWords(data, textField):
    from nltk.corpus import stopwords
    stop = nltk.corpus.stopwords.words('english')
    data[textField] = data[textField].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    return data

def tokenise(data, textField):
    nltk.download('punkt')
    data[textField] = data[textField].apply(lambda x: nltk.tokenize.word_tokenize(x))
    return data

def wordStemmer(text):
    stem_text = [nltk.stem.PorterStemmer().stem(i) for i in text]
    return stem_text

def stemmingText(data, textField):
    data[textField] = data[textField].apply(lambda x: wordStemmer(x))
    return data

def normalizeData(fileLoc, textField):
    data = readData(fileLoc)
    data = removeRowsWithNanDescription(data, textField)
    data = changeUpperCaseCharsWithLower(data, textField)
    data = removeStopWords(data, textField)
    data = removeSymbolsAnsNums(data, textField)
    data = tokenise(data, textField)
    data = stemmingText(data, textField)
    # data = lemmatizeText(data, textField)
    return data

قاعده بیزین:
$$ P(C\mid X) = \frac{P(X \mid C) \, P(C)}{P(X)} $$

Posterior P(c|x): به چه احتمالی یک خبر به این کتگوری تعلق دارد. برای هر خبر به ازای هر کتگوری چک میکنیم که این خبر با چه احتمالی در آن کتگوری قرار دارد. (به کمک محاسبه ی احتمال وجود کلمات خبر در آن کتگوری)
قرار دارد a در کتگوری x یعنی  با چه احتمالی خبر 

Likelihood P(x|c): است. که برای محاسبه آن به ازای هر کلمه در خبر احتمال آن کلمه a با چه احتمالی شامل خبر c یعنی کتگوری
در کتگوری را در احتمال حساب شده تا آن مرحله ضرب می کنیم.
یعنی احتمال وجود کلمات آن خبر در کتگوری

Class Prior P(c): احتمال آن کتگوری. یعنی احتمال وقوع آن کتگوری چقدر است
برای محاسبه تعداد کلمات آن کتگوری را به تعداد کلمات کل داده ترین تقسیم می کنیم.

Evidence P(x): است. که چون می خواهیم احتمالات به ازای کتگوری های مختلف را   x  اویدنس رخ داده درواقع احتمال رخ دادن خبر
مقایسه کنیم و این مخرج در تمام آن ها یکی است آن را محاسبه نمیکنیم

در نهایت برای محاسبه 
ابتدا باید احتمال رخداد کلمه های خبر در کتگوری را در هم ضرب می کنیم و هم چنین در احتمال آن کتگوری ضرب کنیم. likelihood 
چون عدد حاصل بسیار کوچک میشود از آن لگاریتم میگیریم و به جمع تبدیل می شود. بنابراین باید لگاریتم ها را با هم جمع کنیم





برای ترین کردن داده ها ۸۰ درصد از هر کتگوری را به صورت رندم انتخاب کرده و به هم کانکت می کنیم. بعد ۲۰ درصد باقی مانده را برای ارزیابی ذخیره میکنیم.

In [ ]:
def prepareTrainigAndTestingData(fileLoc, fieldToNormalize):
    data = normalizeData(fileLoc, fieldToNormalize)
    businessCategory = data.loc[data[CATEGORYCOL] == BUSINESSCATEGORY]
    styleAndBeautyCategory = data.loc[data[CATEGORYCOL] == STYLEANDBEAUTYCATEGORY]
    travelCategory = data.loc[data[CATEGORYCOL] == TRAVELCATEGORY]
    trainingData = pd.concat([businessCategory.sample(frac = 0.8), styleAndBeautyCategory.sample(frac = 0.8), travelCategory.sample(frac = 0.8)])
    testingData = data.drop(trainingData.index)
    return trainingData, testingData

بعد از آماده کردن دیتا ی ترین آن را بنابر کتگوری به ۳ دسته تقسیم می کنیم و برای هر دسته احتمال تکرار هر کلمه را محاسبه میکنیم. با فرمول تقسیم کردن تعداد تکرار های آن کلمه بر تعداد کل کلمات آن کتگوری.

بنابراین کلاس کتگوری را میسازیم و احتمال کلمات و طول کل کلمات و مجموعه کلمات و احتمال رخداد آن کتگوری در دیتای ترین 
و ... را به ازای هر کتگوری ذخیره میکنیم.

برای حالتی که کلمه ی موجود در خبر در کلمات کتگوری نباشد نحوه محاسبه در سوال ۴ آورده شده
و به همین دلیل برای کلماتی که در کتگوری وجود دارند تعداد را از ۲ شروع کردیم که در واقع همه را با یک جمع کرده باشیم.

In [ ]:
class Category:
    def __init__(self, categoyName):
        self.name = categoyName
        self.accuracyPhase1 = 0
        self.precisionPhase1 = 0
        self.recallPhase1 = 0
        self.accuracyPhase2 = 0
        self.precisionPhase2 = 0
        self.recallPhase2 = 0
        self.p = 0
        
    def calculateProbabilityOfWords(self, data):
        self.wordsProbability = {}
        wordsNum = {}
        lenOfDescription = data[FEILD].apply(lambda x: len(x))
        s = 0
        for num in lenOfDescription:
            s += num
        self.lenOfAllWords = s
        self.allWords = set()
        for arr in data[FEILD]:
            for word in arr:
                if word not in self.allWords:
                    wordsNum[word] = 2
                    self.allWords.add(word)
                else:
                    wordsNum[word] += 1

        for x in wordsNum:
            self.wordsProbability[x] = round(wordsNum[x]*100/self.lenOfAllWords, 8)

به ازای تمام خبر های موجود در داده ی تست احتمال آن خبر در تمام کتگوری ها را محاسبه می کنیم و بین آن ها ماکسیمم می گیریم و کتگوری حدس زده شده را در ستونی جدید با نام 
GUESS
ذخیره می کنیم
برای هر دو فاز به صورت جدا
زیرا برای فاز اول باید سطر هایی که کتگوری استایل دارند را حذف کنیم. در غیر این صورت محاسبات مربوط به بخش 
confusion matrix
غلط خواهد شد و بازده کم می شود

In [ ]:
def predictCategory(words, categories):
    probabilities = {}
    s = 0
    for c in categories:
        s += c.lenOfAllWords
    for c in categories:
        c.p = round((c.lenOfAllWords * 100/s), 8)
    for category in categories:
        probabilityForWordsNotExistInCategory = round(100/category.lenOfAllWords, 8)
        probabilities[category.name] = math.log10(category.p)
        for word in words:
            if word in category.allWords:
                probabilities[category.name] += math.log10(category.wordsProbability[word])
            else:
                probabilities[category.name] += math.log10(probabilityForWordsNotExistInCategory)
    chosenCategory = ''
    maximum = -sys.maxsize - 1
    for p in probabilities:
        if probabilities[p] > maximum:
            maximum = probabilities[p]
            chosenCategory = p
    return chosenCategory

def predictForAllNewsPhase1():
    categoriesPhase1, categoriesPhase2, testingData = calculateProbabilityOfEachWordInCategorys(FEILD, False, False)
    testingData = testingData[~(testingData[CATEGORYCOL] == STYLEANDBEAUTYCATEGORY)]
    testingData['GUESS'] = testingData[FEILD].apply(lambda x: predictCategory(x, categoriesPhase1))
    return categoriesPhase1, categoriesPhase2, testingData

def predictForAllNewsPhase2():
    categoriesPhase1, categoriesPhase2, testingData = calculateProbabilityOfEachWordInCategorys(FEILD, False, False)
    testingData['GUESS'] = testingData[FEILD].apply(lambda x: predictCategory(x, categoriesPhase2))
    return categoriesPhase2, testingData

The Confusion matrix is one of the most intuitive and easiest (unless of course, you are not confused)metrics used for finding the correctness and accuracy of the model. It is used for Classification problem where the output can be of two or more types of classes.

Before diving into what the confusion matrix is all about and what it conveys, Let’s say we are solving a classification problem where we are predicting whether a person is having cancer or not.

Let’s give a label of to our target variable:
1: When a person is having cancer 0: When a person is NOT having cancer.
Terms associated with Confusion matrix:

1. True Positives (TP): True positives are the cases when the actual class of the data point was 1(True) and the predicted is also 1(True)

Ex: The case where a person is actually having cancer(1) and the model classifying his case as cancer(1) comes under True positive.

2. True Negatives (TN): True negatives are the cases when the actual class of the data point was 0(False) and the predicted is also 0(False

Ex: The case where a person NOT having cancer and the model classifying his case as Not cancer comes under True Negatives.

3. False Positives (FP): False positives are the cases when the actual class of the data point was 0(False) and the predicted is 1(True). False is because the model has predicted incorrectly and positive because the class predicted was a positive one. (1)

Ex: A person NOT having cancer and the model classifying his case as cancer comes under False Positives.

4. False Negatives (FN): False negatives are the cases when the actual class of the data point was 1(True) and the predicted is 0(False). False is because the model has predicted incorrectly and negative because the class predicted was a negative one. (0)

Ex: A person having cancer and the model classifying his case as No-cancer comes under False Negatives.


source : https://medium.com/thalus-ai/performance-metrics-for-classification-problems-in-machine-learning-part-i-b085d432082b


<img src="b.png">

به ازای هر دو فاز با فرمول های زیر داریم:

$$ ACCURACY = \frac{TP + TN}{TP + FP + FN + PN} $$

$$ PRECISION = \frac{TP}{TP + FP} $$

$$ RECALL = \frac{TP}{TP + FN} $$


In [ ]:
def evaluationPhase1():
    categoriesPhase1, categoriesPhase2, predictedData = predictForAllNewsPhase1()
    for category in categoriesPhase1:
        tp = len(predictedData[(predictedData[CATEGORYCOL] == category.name) & (predictedData["GUESS"] == category.name)])
        tn = len(predictedData[(predictedData[CATEGORYCOL] != category.name) & (predictedData["GUESS"] != category.name)])
        fp = len(predictedData[(predictedData[CATEGORYCOL] != category.name) & (predictedData["GUESS"] == category.name)])
        fn = len(predictedData[(predictedData[CATEGORYCOL] == category.name) & (predictedData["GUESS"] != category.name)])
        category.accuracyPhase1 = (tp + tn) / (tp + tn + fp + fn)
        category.precisionPhase1 = tp / (tp + fp)
        category.recallPhase1 = tp / (tp + fn)
    return categoriesPhase1

def predictForAllNewsPhase2():
    categoriesPhase1, categoriesPhase2, testingData = calculateProbabilityOfEachWordInCategorys(FEILD, False, False)
    testingData['GUESS'] = testingData[FEILD].apply(lambda x: predictCategory(x, categoriesPhase2))
    return categoriesPhase2, testingData

def evaluationPhase2():
    categoriesPhase2, predictedData = predictForAllNewsPhase2()
    for category in categoriesPhase2:
        tp = len(predictedData[(predictedData[CATEGORYCOL] == category.name) & (predictedData["GUESS"] == category.name)])
        tn = len(predictedData[(predictedData[CATEGORYCOL] != category.name) & (predictedData["GUESS"] != category.name)])
        fp = len(predictedData[(predictedData[CATEGORYCOL] != category.name) & (predictedData["GUESS"] == category.name)])
        fn = len(predictedData[(predictedData[CATEGORYCOL] == category.name) & (predictedData["GUESS"] != category.name)])
        category.accuracyPhase2 = (tp + tn) / (tp + tn + fp + fn)
        category.precisionPhase2 = tp / (tp + fp)
        category.recallPhase2 = tp / (tp + fn)
    return categoriesPhase2

categoryPhase1 = evaluationPhase1()
for c in categoryPhase1:
    print(c.name)
    print("accuracy ", c.accuracyPhase1)
    print("precision ", c.precisionPhase1)
    print("recall ", c.recallPhase1)

categoryFinalPhase2 = evaluationPhase2()
for c in categoryFinalPhase2:
    print(c.name)
    print("accuracy ", c.accuracyPhase2)
    print("precision ", c.precisionPhase2)
    print("recall ", c.recallPhase2)

<table style="width:100%;">
    <tr></tr>
    <tr>
        <td>
            <table style="width:100%; border: 1px solid #ddd;">
                <thead>
                    <tr>
                        <th style="text-align: center;">Phase #1</th>
                    </tr>
                <tr>
                    <th style="text-align: center;">Phase 1</th>
                    <th style="text-align: center;">Travel</th>
                    <th style="text-align: center;">Business</th>
                </tr>
                </thead>
                <tr>
                    <td style="text-align: center;"><b>Recall</b></td>         
                    <td style="text-align: center;">0.8676122931442081</td>
                    <td style="text-align: center;">0.9135667396061269</td>
                </tr>
                <tr>
                    <td style="text-align: center;"><b>Precision</b></td>        
                    <td style="text-align: center;">0.9489334195216548</td>
                    <td style="text-align: center;">0.7884796978281398</td>
                </tr>        
                <tr>
                    <td style="text-align: center;"><b>Accuracy</b></td>         
                    <td style="text-align: center;">0.8837298541826554</td>
                    <td style="text-align: center;">0.8837298541826554</td>
                </tr>
            </table>
        </td>
        <td>
            <table style="width:100%; border: 1px solid #ddd;">
                <thead>
                    <tr>
                        <th style="text-align: center;">Phase #2</th>
                    </tr>
                <tr>
                    <th style="text-align: center;">Phase 2</th>
                    <th style="text-align: center;">Travel</th>
                    <th style="text-align: center;">Business</th>
                    <th style="text-align: center;">Style & Beauty</th>
                </tr>
                </thead>
                <tr>
                    <td style="text-align: center;"><b>Recall</b></td>         
                    <td style="text-align: center;">0.83392434988179</td>
                    <td style="text-align: center;">0.88402625820568</td>
                    <td style="text-align: center;">0.84380403458213</td>
                </tr>
                <tr>
                    <td style="text-align: center;"><b>Precision</b></td>        
                    <td style="text-align: center;">0.88242651657285</td>
                    <td style="text-align: center;">0.70017331022530</td>
                    <td style="text-align: center;">0.92191435768261</td>
                </tr>        
                <tr>
                    <td style="text-align: center;"><b>Accuracy</b></td>         
                    <td style="text-align: center;">0.89196037779313</td>
                    <td style="text-align: center;">0.89587652614604</td>
                    <td style="text-align: center;">0.90900714121170</td>
                </tr>
            </table>
        </td>
    </tr>
    <tr></tr>

<img src="d.png">

confusion matrix for phase 2:

<img src="f.png">

<img src="a.png">

برای حل مشکل 
oversampling:
وقتی از هر کتگوری ۸۰ درصد را برمیداریم و با هم کانکت می کنیم و داده ی ترین را می سازیم باعث می شود که تعداد خبر های هر دسته با هم متفاوت باشند. بنابراین باعث می شود که در محاسبه ی احتمالات تعداد کلمات هر دسته شانس برابری نداشته باشند (یعنی یک دسته به دلیل زیاد بودن تعداد خبر هایش احتمال کلماتش از بقیه دسته ها بیشتر می شود.) بنابر این اندازه ی تمام دسته ها را به بزرگ ترین دسته می رسانیم.
برای این کار از بین خبر های هر دسته به صورت رندم انتخاب می کنیم و با خودش کانکت می کنیم و در واقع در خودش کپی می کنیم. و با این روش باعث میشویم که تفاوت در 
precision and recall
کم شود.


In [ ]:
def dataForOversamplingPase1(fileLoc, fieldToNormalize):
    data = normalizeData(fileLoc, fieldToNormalize)
    businessCategory = data.loc[data[CATEGORYCOL] == BUSINESSCATEGORY]
    travelCategory = data.loc[data[CATEGORYCOL] == TRAVELCATEGORY]
    businessCategory = businessCategory.sample(frac = 0.8)
    travelCategory = travelCategory.sample(frac = 0.8)
    difference = abs(len(businessCategory) - len(travelCategory))
    business = False
    travel = False
    if len(businessCategory) < len(travelCategory):
        lowerOne = businessCategory
        business = True
    else:
        lowerOne = travelCategory
        travel = True
    indexes = lowerOne.index
    temp = lowerOne
    while difference > 0:
        rand = random.choice(indexes)
        row = temp.loc[[rand]]
        lowerOne = pd.concat([lowerOne, row])
        difference-=1
    if business:
        businessCategory = lowerOne
    elif travel:
        travelCategory = lowerOne
    trainingData = pd.concat([businessCategory, travelCategory])    
    testingData = data.drop(trainingData.index)
    testingData = testingData[~(testingData[CATEGORYCOL] == STYLEANDBEAUTYCATEGORY)]
    return trainingData, testingData

def oversamplingPase1():
    categoriesPhase1, categoriesPhase2, testingData = calculateProbabilityOfEachWordInCategorys(FEILD, True, False)
    testingData['GUESS'] = testingData[FEILD].apply(lambda x: predictCategory(x, categoriesPhase1))
    for category in categoriesPhase1:
        tp = len(testingData[(testingData[CATEGORYCOL] == category.name) & (testingData["GUESS"] == category.name)])
        tn = len(testingData[(testingData[CATEGORYCOL] != category.name) & (testingData["GUESS"] != category.name)])
        fp = len(testingData[(testingData[CATEGORYCOL] != category.name) & (testingData["GUESS"] == category.name)])
        fn = len(testingData[(testingData[CATEGORYCOL] == category.name) & (testingData["GUESS"] != category.name)])
        category.accuracyPhase1 = (tp + tn) / (tp + tn + fp + fn)
        category.precisionPhase1 = tp / (tp + fp)
        category.recallPhase1 = tp / (tp + fn)
    return categoriesPhase1

categoryPhase1 = oversamplingPase1()
print("oversampling phase 1")
for c in categoryPhase1:
    print(c.name)
    print("accuracy ", c.accuracyPhase1)
    print("precision ", c.precisionPhase1)
    print("recall ", c.recallPhase1)

def dataForOversamplingPase2(fileLoc, fieldToNormalize):
    data = normalizeData(fileLoc, fieldToNormalize)
    businessCategory = data.loc[data[CATEGORYCOL] == BUSINESSCATEGORY]
    travelCategory = data.loc[data[CATEGORYCOL] == TRAVELCATEGORY]
    styleAndBeautyCategory = data.loc[data[CATEGORYCOL] == STYLEANDBEAUTYCATEGORY]
    businessCategory = businessCategory.sample(frac = 0.8)
    travelCategory = travelCategory.sample(frac = 0.8)
    styleAndBeautyCategory = styleAndBeautyCategory.sample(frac = 0.8)
    business = True
    travel = True
    style = True
    lenArr = [len(businessCategory), len(travelCategory), len(styleAndBeautyCategory)]
    maxLen = max(lenArr)
    smallerCategories = []
    if len(businessCategory) != maxLen:
        business = False
        smallerCategories.append(businessCategory)
    if len(travelCategory) != maxLen:
        travel = False
        smallerCategories.append(travelCategory)
    if len(styleAndBeautyCategory) != maxLen:
        style = False
        smallerCategories.append(styleAndBeautyCategory)
    if business:
        trainingData = businessCategory
    if travel:
        trainingData = travelCategory
    if style:
        trainingData = styleAndBeautyCategory
    for category in smallerCategories:
        indexes = category.index
        temp = category
        while len(category) < maxLen:
            rand = random.choice(indexes)
            row = temp.loc[[rand]]
            category = pd.concat([category, row])
        trainingData = pd.concat([trainingData, category])
    
    testingData = data.drop(trainingData.index)
    print(len(testingData))
    return trainingData, testingData

def oversamplingPase2():
    categoriesPhase1, categoriesPhase2, testingData = calculateProbabilityOfEachWordInCategorys(FEILD, False, True)
    testingData['GUESS'] = testingData[FEILD].apply(lambda x: predictCategory(x, categoriesPhase2))
    for category in categoriesPhase2:
        print(category.name)
        tp = len(testingData[(testingData[CATEGORYCOL] == category.name) & (testingData["GUESS"] == category.name)])
        tn = len(testingData[(testingData[CATEGORYCOL] != category.name) & (testingData["GUESS"] != category.name)])
        fp = len(testingData[(testingData[CATEGORYCOL] != category.name) & (testingData["GUESS"] == category.name)])
        fn = len(testingData[(testingData[CATEGORYCOL] == category.name) & (testingData["GUESS"] != category.name)])
        category.accuracyPhase2 = (tp + tn) / (tp + tn + fp + fn)
        category.precisionPhase2 = tp / (tp + fp)
        category.recallPhase2 = tp / (tp + fn)
    return categoriesPhase2

categoryPhase2 = oversamplingPase2()
print("oversampling phase 2")
for c in categoryPhase2:
    print(c.name)
    print("accuracy ", c.accuracyPhase2)
    print("precision ", c.precisionPhase2)
    print("recall ", c.recallPhase2)

<img src="g.png">

<img src="h.png">

در مرحله آخر برای خبر های موجود در فایل تست نوع آن ها را حدس میزنیم و در یک فایل جدید نتیجه را ذخیره می کنیم

In [ ]:
def test(categoryFinalPhase2):
    data = normalizeData(TESTFILELOCATION, FEILD)
    data['category'] = data[FEILD].apply(lambda x: predictCategory(x, categoryFinalPhase2))
    data = data[['index', 'category']]
    data.drop(data.columns[1], axis=1)
    data.to_csv('out.csv')
    print(data, file = out)

درنرمال کردن با لمتایز فقط باید تابع مورد استفاده در نرمال کردن خبر ها را تغییر دهیم. اگر این کار را انجام دهیم داریم :

In [ ]:
def nltk2wnTag(nltkTag):
    if nltkTag.startswith('J'):
        return nltk.corpus.wordnet.ADJ
    elif nltkTag.startswith('V'):
        return nltk.corpus.wordnet.VERB
    elif nltkTag.startswith('N'):
        return nltk.corpus.wordnet.NOUN
    elif nltkTag.startswith('R'):
        return nltk.corpus.wordnet.ADV
    else:                    
        return None

def wordLemmatizer(text):
    nltkTagged = nltk.pos_tag(nltk.word_tokenize(text))    
    wnTagged = map(lambda x: (x[0], nltk2wnTag(x[1])), nltkTagged)

    resWords = []
    for word, tag in wnTagged:
        if tag is None:                        
            resWords.append(word)
        else:
            resWords.append(nltk.stem.WordNetLemmatizer().lemmatize(word, tag))

    return resWords

def lemmatizeText(data, textField):
    nltk.download('averaged_perceptron_tagger')
    data[textField] = data[textField].apply(lambda x: wordLemmatizer(x))
    return data

def normalizeData(fileLoc, textField):
    data = readData(fileLoc)
    data = removeRowsWithNanDescription(data, textField)
    data = changeUpperCaseCharsWithLower(data, textField)
    data = removeStopWords(data, textField)
    data = removeSymbolsAnsNums(data, textField)
    data = lemmatizeText(data, textField)
    return data

<img src="wo.png">

<img src="w.png">

سوالات:

1- Stemming and Lemmatization both generate the root form of the inflected words. The difference is that stem might not be an actual word whereas, lemma is an actual language word.

Stemming follows an algorithm with steps to perform on the words which makes it faster. Whereas, in lemmatization, you used WordNet corpus and a corpus for stop words as well to produce lemma which makes it slower than stemming. You also had to define a parts-of-speech to obtain the correct lemma.

Stemming usually refers to a crude heuristic process that chops off the ends of words in the hope of achieving this goal correctly most of the time, and often includes the removal of derivational affixes. Lemmatization usually refers to doing things properly with the use of a vocabulary and morphological analysis of words, normally aiming to remove inflectional endings only and to return the base or dictionary form of a word, which is known as the lemma .

stemmers are typically easier to implement and run faster, and the reduced accuracy may not matter for some applications.

Lemmatisation is closely related to stemming. The difference is that a stemmer operates on a single word without knowledge of the context, and therefore cannot discriminate between words which have different meanings depending on part of speech.

For instance:

1-    The word "better" has "good" as its lemma. This link is missed by stemming, as it requires a dictionary look-up.

2-    The word "walk" is the base form for word "walking", and hence this is matched in both stemming and lemmatisation.

3-    The word "meeting" can be either the base form of a noun or a form of a verb ("to meet") depending on the context, e.g., "in our last meeting" or "We are meeting again tomorrow". Unlike stemming, lemmatisation can in principle select the appropriate lemma depending on the context.


در نتیجه استمینگ سریعتره و بیاده سازی آن راحت تر است ولی لزوما کلمه های آن معنی دار نیستند و فقط به ریشه بر می گرداند. ولی لمتایز همه کلمه هاش معنی دارن و کند تره.
لمتایز با توجه به متن تصمیم میگیره و همچنین قابلیت تمیم گیری برای اینکه چه نقش کلماتی رو تغییر بده رو داره

source :
https://www.datacamp.com/community/tutorials/stemming-lemmatization-python and https://stackoverflow.com/questions/1787110/what-is-the-difference-between-lemmatization-vs-stemming

2-

TF-IDF (term frequency-inverse document frequency) is a statistical measure that evaluates how relevant a word is to a document in a collection of documents. This is done by multiplying two metrics: how many times a word appears in a document, and the inverse document frequency of the word across a set of documents.

TF-IDF was invented for document search and information retrieval. It works by increasing proportionally to the number of times a word appears in a document, but is offset by the number of documents that contain the word. So, words that are common in every document, such as this, what, and if, rank low even though they may appear many times, since they don’t mean much to that document in particular.

However, if the word Bug appears many times in a document, while not appearing many times in others, it probably means that it’s very relevant. For example, if what we’re doing is trying to find out which topics some NPS responses belong to, the word Bug would probably end up being tied to the topic Reliability, since most responses containing that word would be about that topic.

TF-IDF for a word in a document is calculated by multiplying two different metrics:

1-    The term frequency of a word in a document. There are several ways of calculating this frequency, with the simplest being a raw count of instances a word appears in a document. Then, there are ways to adjust the frequency, by length of a document, or by the raw frequency of the most frequent word in a document.

2-    The inverse document frequency of the word across a set of documents. This means, how common or rare a word is in the entire document set. The closer it is to 0, the more common a word is. This metric can be calculated by taking the total number of documents, dividing it by the number of documents that contain a word, and calculating the logarithm.

So, if the word is very common and appears in many documents, this number will approach 0. Otherwise, it will approach 1.

<img src="tf.png">


در واقع در این روش هر داکیومنت مثل یک کتگوری در مساله ماست. و به ازای هر کلمه تعداد تکرار آن را با یک جمع می کند و لگاریتم میگیرد. که این معادل لایکلیهود در روش ماست و همچنین بارامتر دومی که در آن ضرب کرده هم لگاریتم احتمال کتگوری است که محاسبه کردیم

بنابراین گر بخواهیم از این روش در بیزین استفاده کنیم باید متریک اول را به جای لایکلیهود و متریک دوم را به جای 
class perior 
قرار دهیم


source : https://monkeylearn.com/blog/what-is-tf-idf/

3-
به عنوان مثال یک بیمارستان را در نظر بگیرید که در تشخیص سرماخوردگی خیلی خوب است و چون تعداد کسانی که سرما خورده اند زیاد است بس کیس های 
زیاد است. در مقابل نمیتواند بیماری ایدز را به درستی تشخیص دهد و اکثر کیس ها را منفی در نظر می گیردtp
در نتیجه دقت آن زیاد است اما از آن جایی که در مجموع در تشخیص دقت بالایی نداریم و کیس هایی که واقعا ایدز ندارند هم زیادند اما ما اکثر آن ها را گفتیم که ندارند بس دقت کمی داریم و عملکرد خوبی نداریم



۴-
اگر کلمه ای در متن خبر باشد که در کلمات یک کتگوری نباشد به جای احتمال آن :
۱/تعداد کل کلمات کتگوری 
قرار میدهیم

و اگر کلمه در یکی از دسته ها یک بار آمده باشد به جای احتمال آن 
۲/تعداد کل کلمات کتگوری 
قرار می دهیم

بدین ترتیب در حالت اول لگاریتم احتمال عدد منفی کوچک تری نسبت به لگاریتم احتمال حالت دوم است. بنابراین در مجموع کتگوری ای که کلمه یک بار در آن آمده است احتمال بیشتری می گیرد و انتخاب شدن آن محتمل تر است (با توجه به احتمال دیگر کلمات خبر)و اگر احتمال بقیه کلمات خبر یکسان باشد بس حتما کتگوری شامل آن کلمه چون احتمال بیشتری دارد انتخاب می شود


